In [0]:
%sql
create schema if not exists ben_experiments 

In [0]:
%sql
use database ben_experiments

In [0]:
%sql
CREATE TABLE if not exists customers (
 id INT,
 age INT,
 salary INT
 )

In [0]:
%sql
insert into customers values 
(1, 19, 20000),
(2, 34, 30000),
(3, 43, 50000),
(4, 30, 2000),
(5, 87, 10000),
(6, 40, 140000),
(7, 43, 50000),
(8, 30, 2000),
(9, 87, 10000),
(10, 40, 140000),
(11, 19, 210000),
(12, 34, 130000),
(13, 43, 150000),
(14, 30, 12000),
(15, 87, 10000),
(16, 40, 140000)


num_affected_rows,num_inserted_rows
16,16


In [0]:
%sql
create table network (
network_id INT,
failure boolean
)

In [0]:
%sql
insert into table network values 
(1, true),
(2, false)

num_affected_rows,num_inserted_rows
2,2


In [0]:
from databricks.feature_store.client import FeatureStoreClient
from databricks.feature_store.entities.feature_lookup import FeatureLookup

fs = FeatureStoreClient()

In [0]:
fs.register_table(
  delta_table='ben_experiments.customers',
  primary_keys='id',
  description='Customer features'
)

2022/07/12 09:29:59 WARNING databricks.feature_store.client: The feature table "ben_experiments.customers" already exists. Use "FeatureStoreClient.write_table" API to write to the feature table.
Out[2]: <FeatureTable: keys=['id'], tags={}>

In [0]:
fs.register_table(
  delta_table='ben_experiments.network',
  primary_keys='network_id',
  description='network features'
)

Out[9]: <FeatureTable: keys=['network_id']>

In [0]:
%sql
create table ground_truth (
customer int,
network int,
label boolean)

In [0]:
%sql
insert into table ground_truth values
(1, 1, true),
(2, 1, false),
(3, 2, true),
(4, 1, false),
(5, 1, true),
(6, 1, false),
(7, 2, true),
(8, 1, false),
(9, 1, true),
(10, 1, false),
(11, 2, true),
(12, 1, false),
(13, 1, true),
(14, 1, false),
(15, 2, true),
(16, 1, false);

num_affected_rows,num_inserted_rows
16,16


In [0]:
df = spark.sql('select * from ground_truth')

In [0]:
feature_lookups = [
    FeatureLookup(
        table_name="ben_experiments.network",
        feature_names=["failure"],
        lookup_key="network"
    ),
    FeatureLookup(
        table_name="ben_experiments.customers",
        feature_names=["salary", "age"],        
        lookup_key="customer")
      
]

training_set = fs.create_training_set(
    df,
    feature_lookups=feature_lookups,
    label="label",
)
training_df = training_set.load_df()

In [0]:
training_df.write.saveAsTable('ben_experiments.customer_churn_train')

In [0]:
from databricks import automl
automl.classify(
  dataset = training_df,
  target_col= 'label',
  max_trials= 10,                             
  primary_metric = "f1",
  )

2022/07/12 09:39:42 INFO databricks.automl.supervised_learner: AutoML run for classification started with run_id: a1bd95b3-4a96-4ad5-bd54-6a2d125dd5dc
2022/07/12 09:39:42 WARNING databricks.automl.supervised_learner: Parameter max_trials is deprecated and has no effect. The parameter will be removed in a future Databricks Runtime release. Chose timeout_minutes to control the duration of the AutoML runs. AutoML stops will automatically stop tuning models if the validation metric no longer improves.
2022/07/12 09:39:43 INFO databricks.automl.base_learner: AutoML run for classification will record all trials to MlFlow experiment with id: 3896267307465297
2022/07/12 09:39:46 WARNING databricks.automl.preprocess: Semantic type `categorical` detected for columns `network`, `salary`, `age`. Training notebooks will encode features based on categorical transformations.
2022/07/12 09:40:00 INFO databricks.automl.supervised_learner: Data exploration complete. Notebook can be found at: https://e2-